In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2

In [2]:
from dynamics.model.dms import DMSWrapper
from dynamics.model.pbmp import PBMP
from dynamics.data.datasets.greener.datamodule import GreenerDataModule
import torch
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint

import hydra
from logging import log
from omegaconf import DictConfig, OmegaConf

In [3]:
hydra.initialize('../config')
hydra.main('../config')

<function hydra.main.main.<locals>.main_decorator(task_function: Callable[[Any], Any]) -> Callable[[], NoneType]>

In [4]:
config = hydra.compose("main")

/home/cch57/.conda/envs/masif/lib/python3.9/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'main': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
config

{'dataset': {'name': 'greener', 'dir': '/home/cch57/projects/protein_dynamics/dynamics/data/datasets/greener', 'fraction': 0.01}, 'model': {'name': 'pbmp', 'params': {'temperature': 0.05, 'timestep': 0.05, 'n_steps': 30, 'k': 20}}, 'name': 'test', 'device': 'cuda', 'training': {'loss': 'rmsd', 'epochs': 10, 'batch_size': 1, 'lr': 0.005, 'logging_freq': 100, 'val_check_interval': 0.5}, 'simulator': {'temperature': 0.05, 'n_steps': 20, 'time_step': 0.05, 'k': 20}}

In [8]:
model = PBMP(**config.model.params).to('cuda')

datamodule = GreenerDataModule(config.dataset.dir, config.training.batch_size, config.dataset.fraction)
data = datamodule.train[0].to('cuda')

In [12]:
model.n_steps = 200

In [13]:
out = model(data.native_coords, data.x, data.res_numbers, data.masses, data.seq)

In [ ]:
model.n_steps

30

In [ ]:
from random import shuffle
from tqdm import tqdm
import os

# PyTorch
import torch
import torch.nn as nn
from torch.nn.functional import normalize

from dynamics.data.datasets.greener.variables import *
from dynamics.model.layers.linear import MLP, ResNet


from dynamics.model.utils.geometric import knn

class DistanceForces(nn.Module):
	"""
	Calculates forces between two atoms based on their 
		1. atoms types
		2. Euclidian distance
		3. Seperation along the sequence
	Input dim = 50 (24*2 + 2)
	Output dim = 1 (a scalar force)
	"""
	def __init__(self, input_size, hidden_size, num_hidden_layers, output_size):
		super(DistanceForces, self).__init__()

		self.model = ResNet(input_size, hidden_size, num_hidden_layers, output_size)

	def forward(self, atom1, atom2, edges):
		messages = torch.cat([atom1+atom2, edges], dim=1)
		return self.model(messages)

class AngleForces(nn.Module):
	"""
	Calculates forces between three atoms making an angle on their 
		1. central atom types
		2. angle around the central atom
	Input dim = 25 (24 + 1)
	Output dim = 1 (a scalar force)
	"""
	def __init__(self, input_size, hidden_size, num_hidden_layers, output_size):
		super(AngleForces, self).__init__()

		self.model = ResNet(input_size, hidden_size, num_hidden_layers, output_size)


	def forward(self, atom1, atom2, atom3, angles):
		messages = torch.cat([atom1, atom2, atom3, angles[:,:,None]], dim=2)
		return self.model(messages)

class DihedralForces(nn.Module):
	"""
	Calculates forces between three atoms making an angle on their 
		1. central atom types
		2. angle around the central atom
	Input dim = 25 (24 + 1)
	Output dim = 1 (a scalar force)
	"""
	def __init__(self, input_size, hidden_size, num_hidden_layers, output_size):
		super(DihedralForces, self).__init__()

		self.model = ResNet(input_size, hidden_size, num_hidden_layers, output_size)

	def forward(self, atom1, atom2, atom3, atom4, dihedrals):
		messages = torch.cat([atom1, atom2, atom3, atom4, dihedrals[:,:,None]], dim=2)
		return self.model(messages)

#----Main model----#
class Simulator(nn.Module):
	def __init__(self, input_size, hidden_size, output_size):
		super(Simulator, self).__init__()

		self.distance_forces = DistanceForces(24+2, 100, 15, 1)
		self.angle_forces = AngleForces(24*3+1, 128, 3, 1)
		self.dihedral_forces = DihedralForces(24*4+1, 128, 5, 1)

	def forward(self, coords, node_f, res_numbers, masses, seq,
				k, n_steps, timestep, temperature, animation, device):

		n_atoms = coords.shape[0]
		n_res = n_atoms // len(atoms)
		model_n = 0

		vels = torch.randn(coords.shape).to(device) * temperature
		accs_last = torch.zeros(coords.shape).to(device)

		for i in range(n_steps):

			coords = coords + vels * timestep + 0.5 * accs_last * timestep * timestep

			idx = knn(coords, k+1)
			senders = idx[:,0].repeat_interleave(k)
			receivers = idx[:,1:].reshape(n_atoms*k)

			# Calc Euclidian distance
			diffs = coords[senders] - coords[receivers]
			dists = diffs.norm(dim=1)
			norm_diffs = diffs / dists.clamp(min=0.01).unsqueeze(1)

			# Calc sequence seperation
			seq_sep = abs(res_numbers[senders] - res_numbers[receivers])/5
			mask = seq_sep > 1
			seq_sep[mask] = 1

			# Concat edge features
			edges1 = torch.cat([dists.unsqueeze(1)-0.01, seq_sep], dim=1)
			edges2 = torch.cat([dists.unsqueeze(1)+0.01, seq_sep], dim=1)

			# Compute forces using MLP
			forces = 50 * (self.distance_forces(node_f[senders], node_f[receivers], edges1) - self.distance_forces(node_f[senders], node_f[receivers], edges2))

			forces = forces * norm_diffs
			total_forces = forces.view(n_atoms, k, 3).sum(1)
			
			batch_size = 1
			atom_types = node_f.view(batch_size, n_res, len(atoms), 24)
			atom_coords = coords.view(batch_size, n_res, 3 * len(atoms))
			atom_accs = torch.zeros(batch_size, n_res, 3 * len(atoms), device=device)
			# Angle forces
			# across_res is the number of atoms in the next residue, starting from atom_3
			for ai, (atom_1, atom_2, atom_3, across_res) in enumerate(angles):
				# Calc vectors and angle between atoms
				ai_1, ai_2, ai_3 = atoms.index(atom_1), atoms.index(atom_2), atoms.index(atom_3)
				if across_res == 0:
					ba = atom_coords[:, :  , (ai_1 * 3):(ai_1 * 3 + 3)] - atom_coords[:, :  , (ai_2 * 3):(ai_2 * 3 + 3)]
					bc = atom_coords[:, :  , (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, :  , (ai_2 * 3):(ai_2 * 3 + 3)]
				elif across_res == 1:
					ba = atom_coords[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] - atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)]
					bc = atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)]
				elif across_res == 2:
					ba = atom_coords[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] - atom_coords[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)]
					bc = atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)]
				ba_norms = ba.norm(dim=2)
				bc_norms = bc.norm(dim=2)
				angs = torch.acos((ba * bc).sum(dim=2) / (ba_norms * bc_norms))
				# Get central atom properties
				if ai == 0:
					atom1 = atom_types[:,:,0,:]
					atom2 = atom_types[:,:,1,:]
					atom3 = atom_types[:,:,2,:]
				if ai == 1:
					atom1 = atom_types[:,:-1,1,:]
					atom2 = atom_types[:,:-1,2,:]
					atom3 = atom_types[:,1:,0,:]
				if ai == 2:
					atom1 = atom_types[:,:-1,2,:]
					atom2 = atom_types[:,1:,0,:]
					atom1 = atom_types[:,1:,1,:]
				if ai == 3:
					atom1 = atom_types[:,:,0,:]
					atom2 = atom_types[:,:,1,:]
					atom3 = atom_types[:,:,3,:]
				if ai == 4:
					atom1 = atom_types[:,:,2,:]
					atom2 = atom_types[:,:,1,:]
					atom3 = atom_types[:,:,3,:]

				angle_forces = 50 * (self.angle_forces(atom1, atom2, atom3, angs-0.01) - self.angle_forces(atom1, atom2, atom3, angs+0.01))

				cross_ba_bc = torch.cross(ba, bc, dim=2)
				fa = angle_forces * normalize(torch.cross( ba, cross_ba_bc, dim=2), dim=2) / ba_norms.unsqueeze(2)
				fc = angle_forces * normalize(torch.cross(-bc, cross_ba_bc, dim=2), dim=2) / bc_norms.unsqueeze(2)
				fb = -fa -fc
				if across_res == 0:
					atom_accs[:, :  , (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, :  , (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, :  , (ai_3 * 3):(ai_3 * 3 + 3)] += fc
				elif across_res == 1:
					atom_accs[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] += fc
				elif across_res == 2:
					atom_accs[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] += fc

						# Dihedral forces
			# across_res is the number of atoms in the next residue, starting from atom_4
			for di, (atom_1, atom_2, atom_3, atom_4, across_res) in enumerate(dihedrals):
				ai_1, ai_2, ai_3, ai_4 = atoms.index(atom_1), atoms.index(atom_2), atoms.index(atom_3), atoms.index(atom_4)
				if across_res == 1:
					ab = atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)] - atom_coords[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)]
					bc = atom_coords[:, :-1, (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)]
					cd = atom_coords[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] - atom_coords[:, :-1, (ai_3 * 3):(ai_3 * 3 + 3)]
				elif across_res == 2:
					ab = atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)] - atom_coords[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)]
					bc = atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)]
					cd = atom_coords[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] - atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)]
				elif across_res == 3:
					ab = atom_coords[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)] - atom_coords[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)]
					bc = atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] - atom_coords[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)]
					cd = atom_coords[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] - atom_coords[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)]
				if di == 0:
					atom1 = atom_types[:,:-1,2,:]
					atom2 = atom_types[:,1:,0,:]
					atom3 = atom_types[:,1:,1,:]
					atom4 = atom_types[:,1:,2,:]
				if di == 1:
					atom1 = atom_types[:,:-1,0,:]
					atom2 = atom_types[:,:-1,1,:]
					atom3 = atom_types[:,:-1,2,:]
					atom4 = atom_types[:,1:,0,:]
				if di == 2:
					atom1 = atom_types[:,:-1,1,:]
					atom2 = atom_types[:,:-1:,2,:]
					atom3 = atom_types[:,1:,0,:]
					atom4 = atom_types[:,1:,1,:]
				if di == 3:
					atom1 = atom_types[:,:-1,2,:]
					atom2 = atom_types[:,1:,0,:]
					atom3 = atom_types[:,1:,1,:]
					atom4 = atom_types[:,1:,3,:]
				if di == 4:
					atom1 = atom_types[:,:-1,3,:]
					atom2 = atom_types[:,:-1,1,:]
					atom3 = atom_types[:,:-1,2,:]
					atom4 = atom_types[:,1:,0,:]
				cross_ab_bc = torch.cross(ab, bc, dim=2)
				cross_bc_cd = torch.cross(bc, cd, dim=2)
				bc_norms = bc.norm(dim=2).unsqueeze(2)
				dihs = torch.atan2(
					torch.sum(torch.cross(cross_ab_bc, cross_bc_cd, dim=2) * bc / bc_norms, dim=2),
					torch.sum(cross_ab_bc * cross_bc_cd, dim=2)
				)

				dih_forces = 50 * (self.dihedral_forces(atom1, atom2, atom3, atom4, dihs-0.01) - self.dihedral_forces(atom1, atom2, atom3, atom4, dihs+0.01))

				fa = dih_forces * normalize(-cross_ab_bc, dim=2) / ab.norm(dim=2).unsqueeze(2)
				fd = dih_forces * normalize( cross_bc_cd, dim=2) / cd.norm(dim=2).unsqueeze(2)
				# Forces on the middle atoms have to keep the sum of torques null
				# Forces taken from http://www.softberry.com/freedownloadhelp/moldyn/description.html
				fb = ((ab * -bc) / (bc_norms ** 2) - 1) * fa - ((cd * -bc) / (bc_norms ** 2)) * fd
				fc = -fa - fb - fd
				if across_res == 1:
					atom_accs[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, :-1, (ai_3 * 3):(ai_3 * 3 + 3)] += fc
					atom_accs[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] += fd
				elif across_res == 2:
					atom_accs[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, :-1, (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] += fc
					atom_accs[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] += fd
				elif across_res == 3:
					atom_accs[:, :-1, (ai_1 * 3):(ai_1 * 3 + 3)] += fa
					atom_accs[:, 1: , (ai_2 * 3):(ai_2 * 3 + 3)] += fb
					atom_accs[:, 1: , (ai_3 * 3):(ai_3 * 3 + 3)] += fc
					atom_accs[:, 1: , (ai_4 * 3):(ai_4 * 3 + 3)] += fd


			
			# Calc distance accs
			accs = total_forces/masses.unsqueeze(1)
			# Calc angle accs
			accs += atom_accs.view(n_atoms, 3) / (masses.unsqueeze(1)*100)

			vels = vels + 0.5 * (accs_last + accs) * timestep
			accs_last = accs

			if animation:
				if i % animation == 0:
					model_n += 1
					save_structure(coords[None,:,:], "animation.pdb", seq, model_n)

		return coords

In [ ]:
old_model = Simulator(None, None, None)

In [ ]:
out = old_model(data.native_coords, data.x, data.res_numbers, data.masses, data.seq, 20, 20, 0.05, 0.05, False, 'cpu')

In [ ]:
data = data.to('cuda')
old_model = old_model.to('cuda')

In [ ]:
out = old_model(data.native_coords, data.x, data.res_numbers, data.masses, data.seq, 20, 100, 0.05, 0.05, False, 'cuda')

In [ ]:
out.sum().backward()